In [0]:
import pandas as pd 
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.datasets import fetch_20newsgroups
from sklearn.decomposition import TruncatedSVD
import re
from bs4 import BeautifulSoup
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score
from sklearn import metrics
import nltk

In [0]:
dataset = fetch_20newsgroups(subset='all',shuffle=True, random_state=42,remove=('headers', 'footers', 'quotes'))

In [0]:
dataset.data

In [0]:
df = pd.DataFrame(data = dataset.data,columns=['text'])

In [0]:
df.shape
df.head(1)

In [0]:
stopwords= set(['br', 'the', 'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't"])


In [0]:

def decontracted(phrase):
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)
    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

preprocessed_reviews = []
for sentence in (df['text']):
    sentence = re.sub(r"http\S+", "", sentence)
    sentence = BeautifulSoup(sentence, 'lxml').get_text()
    sentence = decontracted(sentence)
    sentence = re.sub("\S*\d\S*", "", sentence).strip()
    sentence = re.sub('[^A-Za-z]+', ' ', sentence)
    sentence = ' '.join(e.lower() for e in sentence.split() if e.lower() not in stopwords)
    preprocessed_reviews.append(sentence.strip())


In [0]:
len(preprocessed_reviews)
#preprocessed_reviews[0]

In [0]:
#tfidf_vect = TfidfVectorizer()                     #for plain
#tfidf_vect = TfidfVectorizer(sublinear_tf = True)  #for sublinear normalisation
tfidf_vect = TfidfVectorizer(ngram_range=(2,2),max_features=50000)    #for bigrams
#tfidf_vect = TfidfVectorizer(max_df=0.5 )          #for max normalization
tfidf_vector = tfidf_vect.fit_transform(preprocessed_reviews[0:5000])
tfidf_vector.shape


In [0]:
explained = []
svd = TruncatedSVD(n_components =4000)

In [0]:
tfidf_modif = svd.fit_transform(tfidf_vector)

In [0]:
explained.append(svd.explained_variance_ratio_.sum())
print explained
tfidf_modif.shape

In [0]:
import numpy as np

from sklearn.base import BaseEstimator, ClusterMixin
from sklearn.metrics.pairwise import pairwise_kernels
from sklearn.utils import check_random_state


In [0]:
class KernelKMeans(BaseEstimator, ClusterMixin):

    def __init__(self, n_clusters=3,init='k-means++',max_iter=50, tol=1e-3, random_state=None,
                 kernel="rbf", gamma=0.1, degree=3, coef0=1,
                 kernel_params=None, verbose=0):
        self.n_clusters = n_clusters
        self.max_iter = max_iter
        self.tol = tol
        self.random_state = random_state
        self.kernel = kernel
        self.init = init
        self.gamma = gamma
        self.degree = degree
        self.coef0 = coef0
        self.kernel_params = kernel_params
        self.verbose = verbose
   
    def _pairwise(self):
        return self.kernel == "precomputed"

    def _get_kernel(self, X, Y=None):
        if callable(self.kernel):
            params = self.kernel_params or {}
        else:
            params = {"gamma": self.gamma,
                      "degree": self.degree,
                      "coef0": self.coef0}
        return pairwise_kernels(X, Y, metric=self.kernel,
                                filter_params=True, **params)

    def fit(self, X, y=None, sample_weight=None):
        n_samples = X.shape[0]

        K = self._get_kernel(X)

        sw = sample_weight if sample_weight else np.ones(n_samples)
        self.sample_weight_ = sw

        rs = check_random_state(self.random_state)
        self.labels_ = rs.randint(self.n_clusters, size=n_samples)

        dist = np.zeros((n_samples, self.n_clusters))
        self.within_distances_ = np.zeros(self.n_clusters)

        for it in xrange(self.max_iter):
            dist.fill(0)
            self._compute_dist(K, dist, self.within_distances_,
                               update_within=True)
            labels_old = self.labels_
            self.labels_ = dist.argmin(axis=1)

            # Compute the number of samples whose cluster did not change 
            # since last iteration.
            n_same = np.sum((self.labels_ - labels_old) == 0)
            if 1 - float(n_same) / n_samples < self.tol:
                if self.verbose:
                    print "Converged at iteration", it + 1
                break

        self.X_fit_ = X

        return self

    def _compute_dist(self, K, dist, within_distances, update_within):
        """Compute a n_samples x n_clusters distance matrix using the 
        kernel trick."""
        sw = self.sample_weight_

        for j in xrange(self.n_clusters):
            mask = self.labels_ == j

            if np.sum(mask) == 0:
                raise ValueError("Empty cluster found, try smaller n_cluster.")

            denom = sw[mask].sum()
            denomsq = denom * denom

            if update_within:
                KK = K[mask][:, mask]  # K[mask, mask] does not work.
                dist_j = np.sum(np.outer(sw[mask], sw[mask]) * KK / denomsq)
                within_distances[j] = dist_j
                dist[:, j] += dist_j
            else:
                dist[:, j] += within_distances[j]

            dist[:, j] -= 2 * np.sum(sw[mask] * K[:, mask], axis=1) / denom

    def predict(self, X):
        K = self._get_kernel(X, self.X_fit_)
        n_samples = X.shape[0]
        dist = np.zeros((n_samples, self.n_clusters))
        self._compute_dist(K, dist, self.within_distances_,
                           update_within=False)
        return dist.argmin(axis=1)


In [0]:
score= []
gamma = [0.01,0.1,1,10,100]
for g in gamma :
  kernelkmeans = KernelKMeans(init='k-means++', n_clusters=20, random_state =2,gamma = g,max_iter = 100)
  X = kernelkmeans.fit(tfidf_modif)
  labels = X.labels_
  score.append(silhouette_score(tfidf_modif, labels,metric='euclidean' ))


In [0]:
score

In [0]:
kernelkmeans = KernelKMeans(init='k-means++', n_clusters=20, random_state =2,gamma = 0.01)
X = kernelkmeans.fit(tfidf_modif)
labels = X.labels_

In [0]:
dataset.target[0:5000]


In [0]:
labels


In [0]:
print  metrics.adjusted_rand_score(labels, dataset.target[0:5000]);
print metrics.adjusted_mutual_info_score(labels, dataset.target[0:5000]);
print metrics.normalized_mutual_info_score(labels, dataset.target[0:5000]);
print metrics.homogeneity_score(labels, dataset.target[0:5000]);
print metrics.completeness_score (labels, dataset.target[0:5000]);
print metrics.v_measure_score(labels, dataset.target[0:5000]);
print metrics.fowlkes_mallows_score(labels, dataset.target[0:5000]);

In [0]:
from sklearn.mixture import GaussianMixture
sklearn_pca = PCA()
Y = sklearn_pca.fit_transform(tfidf_modif)
gmm = GaussianMixture(n_components=20, covariance_type='full').fit(Y)
gmm_predict = gmm.predict(Y)


In [0]:
print np.unique(gmm_predict)
gmm_predict

In [0]:
print  metrics.adjusted_rand_score(gmm_predict, dataset.target[0:5000]);
print metrics.adjusted_mutual_info_score(gmm_predict, dataset.target[0:5000]);
print metrics.normalized_mutual_info_score(gmm_predict, dataset.target[0:5000]);
print metrics.homogeneity_score(gmm_predict, dataset.target[0:5000]);
print metrics.completeness_score (gmm_predict, dataset.target[0:5000]);
print metrics.v_measure_score(gmm_predict, dataset.target[0:5000]);
print metrics.fowlkes_mallows_score(gmm_predict, dataset.target[0:5000]);